# Suver Lab NWB Conversion and Reader
This notebook demonstrates how to convert Suver Lab data to NWB format and explore the resulting NWB files.

## Ephys Experiments

This Suver Lab data consists of in-vivo whole-cell patch clamp recordings in Drosophila melanogaster, along with video recordings of the animal's antennae, wingbeat data, and sensory input.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from pynwb import read_nwb

### Load and Explore the NWB File

Now that we have converted the data to NWB format, let's load the NWB file and explore its contents.

In [ ]:
# Load the NWB file
# Update this path to match your NWB file location
nwbfile_path = Path("/home/heberto/cohen_project/Sample data/Suver Lab/nwb/antennae free_3-6dpe_24C06_experiment2.nwb")
assert nwbfile_path.exists(), f"NWB file does not exist: {nwbfile_path}"

nwbfile = read_nwb(nwbfile_path)
nwbfile

In [ ]:
# Display general file metadata
print(f"NWB File: {nwbfile.identifier}")
print(f"Session ID: {nwbfile.session_id}")
print(f"Session Description: {nwbfile.session_description}")
print(f"Session Start Time: {nwbfile.session_start_time}")

# Display subject information if available
if nwbfile.subject is not None:
    print("\nSubject Information:")
    print(f"Subject ID: {nwbfile.subject.subject_id if hasattr(nwbfile.subject, 'subject_id') else 'N/A'}")
    print(f"Age: {nwbfile.subject.age if hasattr(nwbfile.subject, 'age') else 'N/A'}")
    print(f"Sex: {nwbfile.subject.sex if hasattr(nwbfile.subject, 'sex') else 'N/A'}")
    print(f"Genotype: {nwbfile.subject.genotype if hasattr(nwbfile.subject, 'genotype') else 'N/A'}")
    print(f"Species: {nwbfile.subject.species if hasattr(nwbfile.subject, 'species') else 'N/A'}")

### Visualize the Stimulus and Response

In [ ]:
# Initialize lists to store data
stimuli = []
responses = []

# Get all intracellular recordings
# Access the intracellular recordings from the NWB file
# The recordings are stored in the 'intracellular_recordings' table
icephys_table_df = nwbfile.intracellular_recordings.to_dataframe()


number_of_frames = len(icephys_table_df)

# Extract patch clamp data directly from the table
for frame_index in range(number_of_frames):
    row = icephys_table_df.iloc[frame_index]
    stimulus = row["stimuli"]["stimulus"]
    response = row["responses"]["response"]
    
    # We exclude the seal tests
    name_stimulus = stimulus.timeseries.name
    if "Seal" in name_stimulus:
        continue
    stimuli.append(stimulus.timeseries)
    responses.append(response.timeseries)
    
# Stack the stimuli as 2D array
stimuli_data = np.vstack([s.data[:] for s in stimuli])
responses_data = np.vstack([r.data[:] for r in responses])

tachometers = [ts for ts in nwbfile.acquisition.values() if "Tachometer" in ts.name]
puffers = [ts for ts in nwbfile.stimulus.values() if "Puffer" in ts.name]

tachometer_data = [ts.data[:] for ts in tachometers]
puffer_data = [ts.data[:] for ts in puffers]


mean_stimulus = np.mean(stimuli_data, axis=0)
mean_response = np.mean(responses_data, axis=0)
mean_tachometer = np.mean(tachometer_data, axis=0)
mean_puffer = np.mean(puffer_data, axis=0)

# Create time vectors (assuming same timebase for all traces)
# Stimulus time vector

rate = stimuli[0].rate
num_samples = len(stimuli[0].data)
time_stimulus = np.arange(num_samples) / rate

rate = responses[0].rate
num_samples = len(responses[0].data)
time_response = np.arange(num_samples) / rate

# Tachometer time vector

rate = tachometers[0].rate
num_samples = len(tachometers[0].data)
time_tachometer = np.arange(num_samples) / rate

# Puffer time vector

rate = puffers[0].rate
num_samples = len(puffers[0].data)
time_puffer = np.arange(num_samples) / rate


# Create figure with 2x2 subplots
fig, axs = plt.subplots(2, 2, figsize=(15, 10))
fig.tight_layout(pad=4.0)

# Define colors
stimulus_color = 'blue'
response_color = 'green'
tachometer_color = 'red'
puffer_color = 'purple'

# Plot 1: Stimulus data (top-left)
alpha_samples = 0.1
linewidth_samples = 0.5
plot_samples = True

if plot_samples:
    for i in range(len(stimuli_data)):
        axs[0, 0].plot(time_stimulus, stimuli_data[i], color=stimulus_color, alpha=alpha_samples, linewidth=linewidth_samples)
axs[0, 0].plot(time_stimulus, mean_stimulus, color=stimulus_color, linewidth=3.0, label='Mean')
axs[0, 0].set_title('Stimulus Traces')
axs[0, 0].set_xlabel('Time')
axs[0, 0].set_ylabel('Stimulus Amplitude')
axs[0, 0].grid(True)
axs[0, 0].legend()

# Plot 2: Response data (top-right)
if plot_samples:
    for i in range(len(responses_data)):
        axs[0, 1].plot(time_response, responses_data[i], color=response_color, alpha=alpha_samples, linewidth=linewidth_samples)
axs[0, 1].plot(time_response, mean_response, color=response_color, linewidth=3.0, label='Mean')
axs[0, 1].set_title('Response Traces')
axs[0, 1].set_xlabel('Time')
axs[0, 1].set_ylabel('Response Amplitude')
axs[0, 1].grid(True)
axs[0, 1].legend()

# Plot 3: Tachometer data (bottom-left)
if plot_samples:
    for i in range(len(tachometer_data)):
        axs[1, 0].plot(time_tachometer, tachometer_data[i], color=tachometer_color, alpha=alpha_samples, linewidth=linewidth_samples)
axs[1, 0].plot(time_tachometer, mean_tachometer, color=tachometer_color, linewidth=3.0, label='Mean')
axs[1, 0].set_title('Tachometer Traces')
axs[1, 0].set_xlabel('Time')
axs[1, 0].set_ylabel('Tachometer Value')
axs[1, 0].grid(True)
axs[1, 0].legend()

# Plot 4: Puffer data (bottom-right)
if plot_samples:
    for i in range(len(puffer_data)):
        axs[1, 1].plot(time_puffer, puffer_data[i], color=puffer_color, alpha=alpha_samples, linewidth=linewidth_samples)
axs[1, 1].plot(time_puffer, mean_puffer, color=puffer_color, linewidth=3.0, label='Mean')
axs[1, 1].set_title('Puffer Traces')
axs[1, 1].set_xlabel('Time')
axs[1, 1].set_ylabel('Puffer Value')
axs[1, 1].grid(True)
axs[1, 1].legend()


### Visualize the Pose Estimation (DeepLabCut Data)

In [ ]:
behavior_module = nwbfile.processing["behavior"]
pose_estimation_container = behavior_module["PoseEstimationLateralFlyLeft_trial10"]
pose_estimation_container

In [ ]:
pose_estimation_series = pose_estimation_container.pose_estimation_series

# Count the number of nodes
num_nodes = len(pose_estimation_series)

# Create a figure with a grid
# 1 row per node, 3 columns (for the three different plots)
fig, axes = plt.subplots(num_nodes, 3, figsize=(18, 5 * num_nodes))

# If there's only one node, make sure axes is a 2D array
if num_nodes == 1:
    axes = np.array([axes])

# Iterate through nodes
for i, node in enumerate(pose_estimation_series):
    pose_estimation = pose_estimation_series[node]
    data = pose_estimation.data[:]
    
    # Get timestamps
    if pose_estimation.timestamps is not None:
        timestamps = pose_estimation.timestamps[:]
    else:
        timestamps = pose_estimation.starting_time + np.arange(data.shape[0]) / pose_estimation.rate
    
    # Plot 1: Pose Estimation (X vs Y)
    axes[i, 0].plot(data[:, 0], data[:, 1], label=node)
    axes[i, 0].set_title(f"Pose Estimation: {node}")
    axes[i, 0].set_xlabel("X Position")
    axes[i, 0].set_ylabel("Y Position")
    axes[i, 0].legend()
    
    # Plot 2: X Position over time
    axes[i, 1].plot(timestamps, data[:, 0], color='blue', alpha=0.5, linewidth=0.5)
    axes[i, 1].set_title(f"Pose Estimation X Position: {node}")
    axes[i, 1].set_xlabel("Timestamps")
    axes[i, 1].set_ylabel("X Position")
    
    # Plot 3: Y Position over time
    axes[i, 2].plot(timestamps, data[:, 1], color='red', alpha=0.5, linewidth=0.5)
    axes[i, 2].set_title(f"Pose Estimation Y Position: {node}")
    axes[i, 2].set_xlabel("Timestamps")
    axes[i, 2].set_ylabel("Y Position")

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
data

## Behavioral Experiments

### Windy Steps

In [ ]:
from pynwb import read_nwb

nwbfile = read_nwb("/home/heberto/cohen_project/Sample data/Suver Lab/nwb/WindySteps_dark_3-5days.nwb")
nwbfile

In [ ]:
nwbfile.trials.to_dataframe()


### Speedy Bars

In [ ]:
from pynwb import read_nwb
nwbfile = read_nwb("/home/heberto/cohen_project/Sample data/Suver Lab/nwb/SpeedyBars_still_5-7days.nwb")
nwbfile

In [ ]:
nwbfile.trials.to_dataframe()

### Coco 

In [ ]:
from pynwb import read_nwb

nwbfile = read_nwb("/home/heberto/cohen_project/Sample data/Suver Lab/nwb/Coco_both_3-5days.nwb")
nwbfile

In [ ]:
nwbfile.trials.to_dataframe()